In [ ]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Need your help in figuring out what I'm doing wrong.

Hi all, This is not the usual kernel you might have expected. I'm pretty much new to CV and I wanted to participate in this competition to learn CV. 

I have been trying out EfficientNet b3 and as a matter of fact other pre-trained variants of EfficientNets as well as other models like ResNet18, 50 too. But one thing is cosistent, were my validation loss does not reduce beyond the first epoch or 2nd epoch, so is my model accuracy too. 
You can find from the plot below (I moved the cell up after running the entire code below). 

I'm actually stuck for quite sometime with this and I am not able to find what's wrong in my code. I have referred other kernels too but could not understand why. 

Since this is my first time with CV, I would really appreciate if anyone could point me out were am I going wrong. Please do provide your suggestions too if there is anything else I should try.

Note: The below plot is only for one of the 5 stratified folds, however it's the same scenario for other folds and any models too. 

In [ ]:
plt.plot(fold_avg_train_loss_history, label='train')
plt.plot(fold_avg_eval_loss_history, label='eval')
plt.legend()

# fold_avg_train_loss_history

In [ ]:
# ! pip3 install ../input/efficientnet-model/dataclasses-0.6-py3-none-any.whl
# ! pip3 install ../input/efficientnet-model/numpy-1.19.4-cp37-cp37m-manylinux2010_x86_64.whl
# ! pip3 install ../input/efficientnet-model/torch-1.7.0-cp37-cp37m-manylinux1_x86_64.whl
# ! pip3 install ../input/efficientnet-model/typing_extensions-3.7.4.3-py3-none-any.whl

In [ ]:
kaggle = True

In [ ]:
if kaggle:
    #!pip install --no-index --find-links ../input/efficientnet-model/efficientnet_pytorch-0.7.0/dist/ efficientnet-pytorch
    !pip3 install timm
else:
#     ! pip3 install efficientnet_pytorch
    ! pip3 install timm
    ! pip3 install albumentations
#     ! pip3 install albumentations
    pass

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import glob

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torch

import matplotlib.pyplot as plt

import random
from tqdm import tqdm

from PIL import Image

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import StratifiedKFold

# from efficientnet_pytorch import EfficientNet

import timm

import albumentations as A
from albumentations import Compose
from albumentations.pytorch import ToTensorV2

import seaborn as sns
import warnings
warnings.filterwarnings(action='ignore')

# Creating configurations

In [ ]:
if kaggle:
    data_label_path_2020 = "/kaggle/input/cassava-leaf-disease-classification/train.csv"
    train_images_2020_path =  "/kaggle/input/cassava-leaf-disease-classification/train_images/"
    test_images_path =  "/kaggle/input/cassava-leaf-disease-classification/test_images/"
    model_save_path = "./"
    
else:
    data_label_path_2020 = "../data/train.csv"
    
    train_images_2020_path = "../data/train_images"
    test_images_path = "../data/test_images"
    model_save_path = "../saved_models/"
          
        
config = {'data_label_path_2020': data_label_path_2020,
          'train_images_path_2020': train_images_2020_path,
          'use_2019': False, 
          
          'test_images_path': test_images_path,
          'resize_dim': (384, 384),
          'train_batch_size': 16, 
          'test_batch_size': 64,
          'num_workers':8, 
          'device':'cuda' if torch.cuda.is_available() else 'cpu', 
          'model_name': 'effnet', 
          'saved_model_folder': model_save_path}

# Reading the input file¶

In [ ]:
# Reading the training dataset-2020
train_df_2020 = pd.read_csv(config['data_label_path_2020'])
train_df_2020['images_path'] = config['train_images_path_2020']
train_df_2020['data_year'] = 2020
train_df_2020['train_images_path'] = train_df_2020.images_path + '/' + train_df_2020.image_id
# train_df_2020 = train_df_2020.sample(100).reset_index(drop=True)

num_targets = len(train_df_2020.label.unique())

train_df = train_df_2020.copy()

# Forming the test dataset
test_df = pd.DataFrame()
test_df['image_id'] = list(os.listdir(config['test_images_path']))
test_df['images_path'] = config['test_images_path']
test_df['test_images_path'] = test_df.images_path + '/' + test_df.image_id

print(train_df.shape, test_df.shape)

In [ ]:
train_df_2020

# Stratified KFold

In [ ]:
train_df_2020['fold'] = -1

total_folds = 5

np.random.seed(100)
kf = StratifiedKFold(n_splits=total_folds, shuffle=True, random_state=100)

for i, (train, test) in enumerate(kf.split(train_df_2020, train_df_2020.label)):
    train_df_2020['fold'][test] = i
    

# Defining the Dataset formation class

In [ ]:
class Dataset():
    def __init__(self, dataframe, img_path, resize_dim, data_type, transpose, transforms=None):
        self.dataframe = dataframe
        self.img_path = img_path
        self.resize_dim = resize_dim
        self.data_type = data_type
        self.transpose = transpose
        self.transforms = transforms
        
    def __len__(self):
        return len(self.dataframe)
    
    def __getitem__(self, idx):
        image = Image.open(self.img_path[idx])

        image = np.array(image)
        
        if self.transforms:
            image = self.transforms(image=image)['image']
        
        if self.data_type  == 'test':
            label = 1
        else:          
            label = self.dataframe['label'][idx]

        return {'image': image,
                'label': torch.tensor(label, dtype=torch.long)}

# Augmentation

In [ ]:
Image_NET_mean=[0.485, 0.456, 0.406]
Image_NET_std=[0.229, 0.224, 0.225]
    
augmentation = {'train': Compose([A.Resize(config['resize_dim'][0], config['resize_dim'][1], p=1.0),
#                                   A.RandomResizedCrop(256, 256, p=1.0),
#                                   A.Transpose(p=0.5),
                                  A.HorizontalFlip(p=0.5),
                                  A.VerticalFlip(p=0.5),
                                  A.RandomRotate90(p=0.5),
                                  A.ShiftScaleRotate(p=0.5),
                                  A.Rotate(p=0.5),
#                                   A.RandomBrightnessContrast(p=0.5),
#                                   A.RandomShadow(p=0.5),
                                  A.Normalize(mean=Image_NET_mean, 
                                              std=Image_NET_std, 
                                              max_pixel_value=255.0,
                                              always_apply=True,
                                              p=1.0),
                                  ToTensorV2(p=1.0),
                                 ], p=1.) , 
                'valid':Compose([A.Resize(config['resize_dim'][0], config['resize_dim'][1], p=1.0),
                                A.Normalize(mean=Image_NET_mean,
                                            std=Image_NET_std,
                                            max_pixel_value=255.0, 
                                            always_apply=True,
                                            p=1.0),
                                ToTensorV2(p=1.0),
                               ], p=1)
               }

In [ ]:
# Getting the data from dataclass:

def get_data(df, resize_dim, data_type, transpose, transforms):
    data_class = Dataset(dataframe=df,
                         img_path= df['train_images_path'].values,
                         resize_dim=resize_dim,
                         data_type=data_type,
                         transpose=transpose,
                         transforms=transforms)
    return data_class



# Defining the DataLoader

In [ ]:
def define_data_loader(data_class, batch_size, shuffle, num_workers):
    images_dataloader = DataLoader(data_class, 
                                   batch_size=batch_size, 
                                   shuffle=shuffle,
                                   num_workers=config['num_workers'])
    
    return images_dataloader


# Forming the Classifer model class

In [ ]:
timm.list_models()

In [ ]:
class Efficientnet_b3_classifier_for_cassava(nn.Module):
    def __init__(self, num_labels):
        super(Efficientnet_b3_classifier_for_cassava, self).__init__()
        self.num_labels = num_labels
        
        ####################
        self.model = timm.create_model('efficientnet_b3', pretrained=True)
        for param in self.model.parameters():
            param.requires_grad = True
        self.last_layer_size = self.model.classifier.in_features #get the in feature of the last layer
        self.model.classifier = nn.Linear(self.last_layer_size, self.num_labels)                


    def forward(self, x):
        output = self.model(x)
        return output
    

In [ ]:
Efficientnet_b3_classifier_for_cassava(num_labels=5)

In [ ]:
criterion = nn.CrossEntropyLoss()

def setting_seed(seed_no):
    random.seed(seed_no)
    np.random.seed(seed_no)
    torch.manual_seed(seed_no)
    torch.cuda.manual_seed_all(seed_no)    

    
def model_saving(model, epoch):    
        model_name = config['saved_model_folder'] + 'best_'+ config['model_name'] + 'epoch_' + str(epoch) + ".bin"
        torch.save(model.state_dict(), model_name)


# Train function

In [ ]:
def train_fn(data_loader, model, optimizer, scheduler, params):
    
    model.train()
    setting_seed(seed_no = seed)
    
    train_loss  = 0
    for index, dataset in tqdm(enumerate(data_loader), total = len(data_loader)):
        image = dataset['image'].to(config['device'], dtype = torch.float)
        target = dataset['label'].to(config['device'], dtype = torch.long)
    
        prediction = model(image)
        
        step_loss = criterion(prediction, target)
        
        step_loss.sum().backward()
        optimizer.step()
#         scheduler.step()
        optimizer.zero_grad()
        
        #torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        
        train_loss += step_loss
        
    
    avg_train_loss = train_loss/len(data_loader)
    
    return avg_train_loss
    

# Eval function

In [ ]:
def eval_fn(data_loader, model):
    
    model.eval()
    eval_loss = 0
    
    actual_output = torch.tensor([]).to(config['device'], dtype=torch.long)
    predicted_prob = torch.tensor([]).to(config['device'], dtype=torch.float)
    
    with torch.no_grad():
        for index, dataset in tqdm(enumerate(data_loader), total = len(data_loader)):
            image = dataset['image'].to(config['device'], dtype = torch.float)
            target = dataset['label'].to(config['device'], dtype = torch.long)
            
            prediction = model(image)
            
            step_loss = criterion(prediction, target)
            eval_loss += step_loss
            
            actual_output = torch.cat((actual_output, target))
            predicted_prob = torch.cat((predicted_prob, prediction))
                        
            #print("Prediction", prediction.shape, prediction)
            #print("predicted_output", predicted_output.shape, predicted_output)
            #print("###########################################")
            
        actual_class = np.array(actual_output.detach().cpu())
        predicted_class = np.argmax(np.array(predicted_prob.detach().cpu()), axis = 1)

        conf_mat = confusion_matrix(actual_class, predicted_class)
        print(conf_mat)

        avg_eval_loss = eval_loss/len(data_loader)        

    return avg_eval_loss, actual_class, predicted_prob,  predicted_class


# Training Engine

In [ ]:
def training_engine(EPOCHS, train_data, valid_data, patience, lr, fold):
       
    setting_seed(seed_no = seed)
    model = Efficientnet_b3_classifier_for_cassava(num_labels=num_targets)
    model = nn.DataParallel(model)
    model.to(config['device'])
    
    optimizer_grouped_parameters = model.parameters() #params_2_tune(model)
    optimizer = torch.optim.AdamW(optimizer_grouped_parameters, lr = lr, 
                                 weight_decay=0.00001)
#     optimizer = torch.optim.SGD(optimizer_grouped_parameters, lr = 0.0005)
    
    total_steps = len(train_data) * EPOCHS
    
    # Set up the learning rate scheduler
    # scheduler = transformers.get_linear_schedule_with_warmup(optimizer,
    #                                                         num_warmup_steps=0, # Default value
    #                                                         num_training_steps=total_steps)

    # This scheduler will be called after the validation loss is calculated.
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2, 
                                                           threshold=0.0001, threshold_mode='rel', cooldown=0, 
                                                           min_lr=0, eps=1e-08, verbose=True)
    
    best_accuracy=0
    counter=0
    
    fold_avg_train_loss_history = []
    fold_avg_eval_loss_history = []
    
    for epoch in range(1, EPOCHS):    
    
        # Training
        avg_train_loss = train_fn(data_loader = train_data,
                                  model = model,
                                  optimizer = optimizer, 
                                  scheduler = scheduler, 
                                  params = optimizer_grouped_parameters)

        # Evaluation
        avg_eval_loss, actual_class, predicted_prob, predicted_class = eval_fn(data_loader = valid_data,
                                                                               model = model)
        
        acc = accuracy_score(actual_class, predicted_class)

        print(f"Fold: {fold}, Epoch: {epoch}/{EPOCHS}, train_loss: {avg_train_loss}, eval_loss {avg_eval_loss}, Accuracy: {acc}")

        if (acc>best_accuracy):
            best_accuracy = acc
            best_eval_loss = avg_eval_loss
            counter = 0
            
            final_epoch = epoch
            
            print('Saving the model')
            model_saving(model, epoch=epoch)
        
        else:
            counter+=1
            print(f"Accuracy did not improve from the best {best_accuracy}, patience is {counter}/{patience}")
            final_epoch = epoch
            
            if counter==patience:
                print(f"Maximum patience level {patience} reached so exiting the training")
                final_epoch = epoch-patience if epoch>patience else epoch
                
                break            
        
        fold_avg_train_loss_history.append(avg_train_loss.item())
        fold_avg_eval_loss_history.append(avg_eval_loss.item())
        
    return model, actual_class, predicted_prob, predicted_class, \
           fold_avg_train_loss_history, fold_avg_eval_loss_history, \
           best_accuracy, best_eval_loss, final_epoch


# K Fold Training

In [ ]:
fold_summary = {}
for i in range(total_folds):
    current_fold = i+1
    train_local = train_df_2020.loc[train_df_2020.fold!=i, :].reset_index(drop=True)
    test_local = train_df_2020.loc[train_df_2020.fold==i, :].reset_index(drop=True)
    
    train_local_images = get_data(df=train_local, 
                                  resize_dim=config['resize_dim'], 
                                  data_type='train', 
                                  transpose=True,
                                  transforms=augmentation['train'])

    test_local_images = get_data(df=test_local,
                                 resize_dim=config['resize_dim'],
                                 data_type='train',
                                 transpose=True,
                                 transforms=augmentation['valid'])
    
    train_local_images_dataloader = define_data_loader(data_class=train_local_images, 
                                                       batch_size=config['train_batch_size'], 
                                                       shuffle=True, 
                                                       num_workers=config['num_workers'])

    test_local_images_dataloader = define_data_loader(data_class=test_local_images, 
                                                      batch_size=config['test_batch_size'], 
                                                      shuffle=False, 
                                                      num_workers=config['num_workers'])

    #print(f"Training the fold {current_fold}")
    # Training on the Local dataset
    seed = 50
    model, actual, predicted_prob, predicted_class, fold_avg_train_loss_history, fold_avg_eval_loss_history, best_accuracy, best_eval_loss, final_epoch = training_engine(EPOCHS=10000, 
                                                                                                                                                                            train_data=train_local_images_dataloader, 
                                                                                                                                                                            valid_data=test_local_images_dataloader, 
                                                                                                                                                                            patience=15, 
                                                                                                                                                                            lr=0.0001,
                                                                                                                                                                            fold=current_fold)
    
    

    fold_summary['fold' + str(i+1)] = f"Fold: {i+1}, Epoch: {final_epoch}, Eval loss : {np.round(best_eval_loss.detach().cpu(), 5)}, Eval accuracy : {np.round(best_accuracy, 5)}"
    
    print("")
    print("######################################################################################")
    print("################################# Fold results #######################################")
    print("######################################################################################")
    for fold_info in fold_summary.keys():
        print(fold_summary[fold_info])
    print("")
    
    break